# 🧠 TinyML - Colab Training Notebook

This notebook trains your TinyML model on Google Colab GPU.

**Flow:**
1. Check GPU
2. Clone or update your GitHub repo
3. Install Colab-compatible dependencies
4. Run training script (`train.py` or `src/train.py`)
5. Save exported model (e.g. `tiny_model.tflite`) to Google Drive

Customize the training script path in the **Run training** cell if needed.


In [ ]:
# 1. 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

# 2. 데이터 폴더 경로 설정
DATA_DIR = "/content/drive/MyDrive/TinyML_models"

## 1️⃣ Runtime & GPU check
Make sure you set **Runtime → Change runtime type → Hardware accelerator → GPU** before running.


In [ ]:
!nvidia-smi || echo "No NVIDIA GPU detected. Please enable GPU in Runtime settings."


## 2️⃣ Clone or update TinyML repository

Set your GitHub repo URL if different.


In [ ]:
import os
import subprocess

REPO_URL = "https://github.com/danielsoo/TinyML.git"  # change if needed
PROJECT_DIR = "/content/TinyML"

if not os.path.exists(PROJECT_DIR):
    subprocess.run(["git", "clone", REPO_URL, PROJECT_DIR], check=True)
else:
    subprocess.run(["git", "-C", PROJECT_DIR, "pull"], check=True)

os.chdir(PROJECT_DIR)
subprocess.run(["ls"], check=True)


## 2️⃣.5 Update config data path
Google Drive에 있는 데이터 경로(`DATA_DIR`)를 `config/federated.yaml`에 반영합니다.



In [ ]:
import yaml
from pathlib import Path

config_path = Path(PROJECT_DIR) / "config" / "federated.yaml"

if config_path.exists():
    with config_path.open("r") as f:
        cfg = yaml.safe_load(f)

    cfg.setdefault("data", {})
    cfg["data"]["path"] = DATA_DIR

    with config_path.open("w") as f:
        yaml.safe_dump(cfg, f, sort_keys=False, allow_unicode=True)

    print(f"Updated federated.yaml data.path -> {cfg['data']['path']}")
else:
    raise FileNotFoundError(f"Cannot find {config_path}")



## 3️⃣ Generate Colab-specific requirements (no macOS-only packages)

We remove `tensorflow-macos` and `tensorflow-metal` from `requirements.txt` automatically.


In [ ]:
import os

src_req = "requirements.txt"
colab_req = "colab_requirements.txt"

# Skip packages that are preinstalled or incompatible on Colab (tensorflow/numpy variants)
skip_keywords = ["tensorflow", "numpy", "tensorflow-macos", "tensorflow-metal"]

if os.path.exists(src_req):
    with open(src_req, "r") as f:
        lines = f.readlines()

    with open(colab_req, "w") as f:
        for line in lines:
            if any(kw in line for kw in skip_keywords):
                continue
            f.write(line)

    print("Generated:", colab_req)
    with open(colab_req, "r") as f:
        print(f.read())
else:
    print("No requirements.txt found. Skipping Colab requirements generation.")


## 4️⃣ Install dependencies (Colab compatible)

- Installs from `colab_requirements.txt` if present.
- Installs standard `tensorflow` for Linux GPU.


In [ ]:
import os
import subprocess

if os.path.exists("colab_requirements.txt"):
    subprocess.run(["pip", "install", "-r", "colab_requirements.txt"], check=True)
else:
    print("colab_requirements.txt not found. Install your packages manually if needed.")

subprocess.run(["pip", "install", "flwr[simulation]"], check=True)

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU devices:", tf.config.list_physical_devices('GPU'))


## 5️⃣ (Optional) Download or prepare dataset

Edit this cell if your training script expects data in a specific path.
For example, you can mount Google Drive or download from Kaggle here.


In [ ]:
# Example: mount Google Drive if your data is stored there.
# from google.colab import drive
# drive.mount('/content/drive')

# Example: create a data directory
# os.makedirs('data', exist_ok=True)
# Then copy or download your dataset into ./data

print("Dataset preparation step: customize as needed.")


## 6️⃣ Run training

This cell tries to run `train.py` at repo root.
If your main script is at a different path, edit accordingly (e.g. `src/train.py`).


In [ ]:
import pandas as pd
from pathlib import Path

data_dir = Path("/content/drive/MyDrive/TinyML_models")  # CSV들이 있는 경로로 수정
csv_paths = sorted(data_dir.glob("*.csv"))

dfs = [pd.read_csv(p, low_memory=False) for p in csv_paths]
df = pd.concat(dfs, ignore_index=True)

print("Total Samples:", len(df))
print(df["attack"].value_counts())
print(df["attack"].value_counts(normalize=True))  # 비율 확인

In [ ]:
import os

PROJECT_DIR = "/content/TinyML"
os.chdir(PROJECT_DIR)

# Default entry point: Flower federated simulation
# Edit the command below if you want to run a different training script.
print("Running python -m src.federated.client --save-model src/models/global_model.h5")
!python -m src.federated.client --save-model src/models/global_model.h5


## 7️⃣ Save trained model(s) to Google Drive

This will look for common output filenames (e.g. `tiny_model.tflite`) in the project root and copy them to your Drive.
Edit `OUTPUT_FILES` if your script uses different names or locations.


In [ ]:
import os
import shutil
from google.colab import drive

PROJECT_DIR = "/content/TinyML"
OUTPUT_FILES = [
    "src/models/global_model.h5",
    "src/models/global_model.tflite",
    "models/global_model.h5",
    "models/global_model.tflite",
    "tiny_model.tflite",
    "model.tflite",
    "model.h5",
    "saved_model.pb"
]

# Mount Drive to store the trained models
drive.mount('/content/drive')

dest_dir = "/content/drive/MyDrive/TinyML_models"
os.makedirs(dest_dir, exist_ok=True)

found_any = False
for fname in OUTPUT_FILES:
    src_path = os.path.join(PROJECT_DIR, fname)
    if os.path.exists(src_path):
        dst_path = os.path.join(dest_dir, fname)
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)
        shutil.copy(src_path, dst_path)
        print(f"Copied {src_path} -> {dst_path}")
        found_any = True

if not found_any:
    print("No known model files found. Make sure your training script saves a model and update OUTPUT_FILES if needed.")
